[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/llamaindex/self-correcting-query-engine/self-correcting.ipynb)

## Configuration 

In [ ]:
# this is the llamaindex used
%pip install -U weaviate-client llama-index

In [1]:
import llama_index
import weaviate
from importlib.metadata import version

print(f"LlamaIndex version: {version('llama_index')}")
print(f"Weaviate client version: {version('weaviate-client')}")

# Needed for running async functions in Jupyter Notebook
import nest_asyncio

nest_asyncio.apply()

LlamaIndex version: 0.11.23
Weaviate client version: 4.9.3


## Connect to Weaviate

In [2]:
import weaviate

# Connect to your Weaviate instance
client = weaviate.connect_to_embedded()

# lets check the connection getting the server version
print(f"Client: {weaviate.__version__}, Server: {client.get_meta().get('version')}")

{"action":"startup","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-11-14T09:57:44-03:00"}
{"action":"startup","auto_schema_enabled":true,"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-11-14T09:57:44-03:00"}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-11-14T09:57:44-03:00"}
{"build_git_commit":"ab0312d5d","build_go_versio

Client: 4.9.3, Server: 1.26.6


{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","docker_image_tag":"localhost","level":"info","msg":"configured versions","server_version":"1.26.6","time":"2024-11-14T09:57:46-03:00"}
{"action":"grpc_startup","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"grpc server listening at [::]:50050","time":"2024-11-14T09:57:46-03:00"}
{"address":"192.168.28.99:57990","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"current Leader","time":"2024-11-14T09:57:46-03:00"}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"starting migration from old schema","time":"2024-11-14T09:57:46-03:00"}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","bu

{"action":"telemetry_push","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"telemetry started","payload":"\u0026{MachineID:02bf4a41-5c36-467a-8d04-f0bfea14a085 Type:INIT Version:1.26.6 NumObjects:0 OS:darwin Arch:arm64 UsedModules:[]}","time":"2024-11-14T09:57:46-03:00"}


## Create Schema

In [3]:
from weaviate import classes as wvc

# lets make sure we do not have a collection with the same name
client.collections.delete("BlogPost")

collection = client.collections.create(
    "BlogPost",
    description="Blog post from the Weaviate website.",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(model="text-embedding-3-small"),
    generative_config=wvc.config.Configure.Generative.openai(model="gpt-3.5-turbo"),
    properties=[
        wvc.config.Property(name="content", data_type=wvc.config.DataType.TEXT)
    ]
)

{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/blogpost/ncpeXSpBjGZf/proplengths does not exist, creating new tracker","time":"2024-11-14T09:57:51-03:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-11-14T09:57:51-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"Created shard blogpost_ncpeXSpBjGZf in 3.014125ms","time":"2024-11-14T09:57:51-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":

## Add Data

In [4]:
from llama_index.core import SimpleDirectoryReader

blogs = SimpleDirectoryReader('./data').load_data()

## Setup Weaviate Vector Store

In [5]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext
import os

from llama_index.core.settings import Settings

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.settings import Settings

# make sure you habe the OPENAI api key
openai_api_key = os.environ["OPENAI_API_KEY"]

Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")


# construct vector store
vector_store = WeaviateVectorStore(weaviate_client = client, index_name="BlogPost", text_key="content")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store = vector_store)

# set up the index
index = VectorStoreIndex.from_documents(blogs, storage_context = storage_context)

## Query without Self-Corrrecting

In [6]:
base_query_engine = index.as_query_engine()
query = "What is Ref2Vec?"

response = base_query_engine.query(query)
print(response)

Ref2Vec is about representing a data object based on the objects it references. The `ref2vec-centroid` module specifically uses the average, or centroid vector, of the cross-referenced vectors to represent the referencing object.


## Configure Self-Correcting Query Engine

In [7]:
from llama_index.core.evaluation import GuidelineEvaluator
from llama_index.core.evaluation.guideline import DEFAULT_GUIDELINES
from llama_index.core import Response
from llama_index.core.indices.query.query_transform.feedback_transform import (
    FeedbackQueryTransformation,
)
from llama_index.core.query_engine import RetryGuidelineQueryEngine

# Guideline eval
guideline_eval = GuidelineEvaluator(
    guidelines=DEFAULT_GUIDELINES
    + "\nThe response should not be overly long.\n"
    "The response should try to summarize where possible.\n"
)  # just for example

In [9]:
typed_response = (
    response if isinstance(response, Response) else response.get_response()
)
eval = guideline_eval.evaluate_response(query, typed_response)
print(f"Guideline eval evaluation result: {eval.feedback}")

feedback_query_transform = FeedbackQueryTransformation(resynthesize_query=True)
transformed_query = feedback_query_transform.run(query, {"evaluation": eval})
print(f"Transformed query: {transformed_query.query_str}")

Guideline eval evaluation result: The response provides a clear explanation of Ref2Vec, mentioning how it represents a data object based on the objects it references. It also explains the specific use of the `ref2vec-centroid` module in calculating the centroid vector. The response could be improved by providing an example or further elaborating on the application of Ref2Vec in a real-world scenario.
Transformed query: Here is a previous bad answer.
Ref2Vec is about representing a data object based on the objects it references. The `ref2vec-centroid` module specifically uses the average, or centroid vector, of the cross-referenced vectors to represent the referencing object.
Here is some feedback from the evaluator about the response given.
The response provides a clear explanation of Ref2Vec, mentioning how it represents a data object based on the objects it references. It also explains the specific use of the `ref2vec-centroid` module in calculating the centroid vector. The response 

In [10]:
typed_response = response if isinstance(response, Response) else response.get_response()
eval = guideline_eval.evaluate_response(query, typed_response)
print(f"Guideline eval evaluation result: {eval.feedback}")

feedback_query_transform = FeedbackQueryTransformation(resynthesize_query=True)
transformed_query = feedback_query_transform.run(query, {"evaluation": eval})
print(f"Transformed query: {transformed_query.query_str}")

Guideline eval evaluation result: The response provides a clear explanation of Ref2Vec, mentioning how it represents a data object based on the objects it references. It also explains the specific use of the `ref2vec-centroid` module in calculating the centroid vector. The response could benefit from providing an example or further elaborating on the practical applications of Ref2Vec.
Transformed query: Here is a previous bad answer.
Ref2Vec is about representing a data object based on the objects it references. The `ref2vec-centroid` module specifically uses the average, or centroid vector, of the cross-referenced vectors to represent the referencing object.
Here is some feedback from the evaluator about the response given.
The response provides a clear explanation of Ref2Vec, mentioning how it represents a data object based on the objects it references. It also explains the specific use of the `ref2vec-centroid` module in calculating the centroid vector. The response could benefit fr

In [11]:
retry_guideline_query_engine = RetryGuidelineQueryEngine(
    base_query_engine, guideline_eval, resynthesize_query=True
)
retry_guideline_response = retry_guideline_query_engine.query(query)
print(retry_guideline_response)

Ref2Vec is about representing a data object based on the objects it references. The `ref2vec-centroid` module specifically uses the average, or centroid vector, of the cross-referenced vectors to represent the referencing object.
